1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
   - 토큰수 초과로 답변을 생성하지 못할 수 있고
   - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달


In [15]:
!uv add -q langchain_text_splitters


In [71]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# overlap: 어느정도 chunk 끼리 겹치게해서 유사도 높게함
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter)

In [72]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [73]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory="./chroma_db",
#     collection_name="chroma-tax",
# )
database = Chroma(
    collection_name="chroma-tax",
    persist_directory="./chroma_db",
    embedding_function=embedding,
)

In [74]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3)

In [75]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

In [76]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [77]:
ai_message = llm.invoke(prompt)

In [78]:
ai_message.content

'연봉 5천만 원인 직장인의 소득세는 여러 요소에 따라 달라지지만, 일반적인 계산 방법을 통해 대략적인 세액을 산출할 수 있습니다. 아래는 기본적인 계산 과정입니다.\n\n1. 근로소득 공제액 산정\n- 근로소득 공제는 소득 금액에 따라 차등 적용됩니다.\n- 5,000만원 이상의 근로소득에 대한 근로소득 공제는 약 1,200만원 수준입니다. (세법 개정에 따라 다소 차이가 있을 수 있음)\n\n2. 과세표준 계산\n- 과세표준 = 총소득(연봉) - 근로소득공제 - 기타 공제(인적공제, 보험공제 등)\n- 여기서는 기타 공제는 고려하지 않고, 근로소득공제만 적용합니다.\n- 과세표준 ≈ 5,000만원 - 1,200만원 = 3,800만원\n\n3. 세율 적용\n- 한국의 소득세는 누진세율 구조로, 과세표준에 따라 세율이 다르게 적용됩니다.\n- 2024년 기준 세율 표는 다음과 같습니다 (일부 예시):\n  - 1,200만원 이하: 6%\n  - 1,200만원 초과 ~ 4,600만원 이하: 15%\n  - 4,600만원 초과 ~ 8,800만원 이하: 24%\n  - 그 이상\n\n4. 세액 계산 예시\n- 3,800만원의 과세표준에 대한 세액은:\n  - 1,200만원까지: 1,200만원 × 6% = 72만원\n  - 나머지 2,600만원(3,800만원 - 1,200만원): 2,600만원 × 15% = 390만원\n  - 총 세액: 72만원 + 390만원 = 약 462만원\n\n5. 추가 공제 및 세액 공제 적용\n- 근로소득 세액공제, 보험료 세액공제, 인적공제 등을 차감하면 최종 납세액이 결정됩니다.\n- 이 예시는 단순 계산이므로, 실제 세금은 각종 공제와 세액공제를 반영하여 다소 차이날 수 있습니다.\n\n결론: 대략 460만 원 내외가 예상되며, 정확한 계산을 위해서는 본인의 상세 공제 내역과 소득 구간별 세율을 반영하는 것이 필요합니다.\n\n추가로 궁금하신 점이나, 세액 계산 시 적용받는 공제 항목에 대해 자세한 안내가 필요하시면 알려주세요.'

In [79]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")


/Users/sys/Desktop/codes/rag-llm-langchain/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [80]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [82]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=database.as_retriever(), chain_type_kwargs={"prompt": prompt}
)

In [85]:
ai_message = qa_chain.invoke({"query": query})

In [86]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 구체적인 세율과 공제 내용이 제공되지 않아 정확한 금액을 알 수 없습니다. 일반적으로 소득세는 과세표준과 세율에 따라 결정되며, 5천만원 소득에는 일정 공제와 세율이 적용됩니다. 따라서 정확한 세액 계산을 위해서는 공제액, 세율, 그리고 기타 소득정보가 필요합니다.'}